## Feature Extraction Notebook

---

This notebook is used to extract features from the data. The audio is divided into windows of 1 second. The features extracted from each audio window are:

- **MFCC (Mel-Frequency Cepstral Coefficients)**: n_mfcc chosen by the user
- **Chroma STFT**: 12 coefficients
- **CQT (Constant-Q Transform)**: n_cqt chosen by the user
- **RMS (Root Mean Square) Energy**: 1 coefficient
- **Spectral Centroid**: 1 coefficient
- **Spectral Bandwidth**: 1 coefficient
- **Spectral Roll-off**: 1 coefficient
- **Zero Crossing Rate**: 1 coefficient

#### Sections:

- [Feature Extraction](#Feature-Extraction)
  - [Specific Features Extraction](#Specific-Features-Extraction)
  - [All Features Extraction](#All-Features-Extraction)

#### Findings:

- We have highly unbalanced classes in the dataset.


In [1]:
# import all the functions
import utils as utils
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
from tqdm.notebook import tqdm
import os
import importlib
import torchaudio
import torch

# import the utils module
importlib.reload(utils)

ModuleNotFoundError: No module named 'librosa'

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

# -------- tqdm DARK THEME --------

### Features Extraction


In [ ]:
# hyperparameters
#sr = 'mix'
sr = 4000
INTERVAL = 2

# set the paths to data
BASE_DIR = "../dataset/"
ARTIFACTS_DIR = BASE_DIR + f"artifacts_{sr}/"
EXTRAHLS_DIR = BASE_DIR + f"extrahls_{sr}/"
MURMURS_DIR = BASE_DIR + f"murmurs_{sr}/"
NORMALS_DIR = BASE_DIR + f"normals_{sr}/"
EXTRASTOLES_DIR = BASE_DIR + f"extrastoles_{sr}/"

# paths to save the features
#FEATURES_RAW_DIR = "../features/raw/"
#FEATURES_RAW_DIR = "../features/balanced/priori/"
FEATURES_RAW_DIR = "../features/balanced/posteriori/"
#FEATURES_RAW_DIR = "../features/balanced/both/"

PATHS = [ARTIFACTS_DIR, EXTRAHLS_DIR, MURMURS_DIR, NORMALS_DIR, EXTRASTOLES_DIR]

# REMOVE THE AUGEMENTED SAMPLES WHEN EXTRACTING BASE RAW FEATURES
for path in PATHS:
     utils.remove_generated_samples(path, 'USERAUGMENTED')

Removed 0 generated samples from ../dataset/artifacts_4000/ directory
Removed 131 generated samples from ../dataset/extrahls_4000/ directory
Removed 0 generated samples from ../dataset/murmurs_4000/ directory
Removed 0 generated samples from ../dataset/normals_4000/ directory
Removed 104 generated samples from ../dataset/extrastoles_4000/ directory


In [2]:
print(enumerate(PATHS))

NameError: name 'PATHS' is not defined

### Specific Features Extraction

- **MFCC (Mel-Frequency Cepstral Coefficients)**: n_mfcc chosen by the user
- **Chroma STFT**: 12 coefficients
- **CQT (Constant-Q Transform)**: n_cqt chosen by the user


In [4]:
def extract_features(
    dir_path: str,
    label: str,
    extraction_interval: float,
    sample_rate: int = 44100,
    n_mfcc: int = 13,
    melkwargs: dict = {},
    n_cqt: int = 84,
    n_rms: int = 20,
    n_zcr: int = 20,
    n_sc: int = 20,
    n_sb: int = 20, 
    n_sr: int = 20,
    get_mfcc: bool = False,
    get_chroma: bool = False,
    get_cqt: bool = False,
    get_rms: bool = False,
    get_zcr: bool = False,
    get_sc: bool = False,
    get_sb: bool = False,
    get_sr: bool = False,
) -> list:
    """
    Extracts audio features (MFCC, chroma, RMS, spectral centroid, spectral bandwidth, spectral rolloff, zero-crossing rate)
    from audio files in the specified directory.

    Args:
        dir_path (str): The path to the directory containing audio files.
        label (str): The label associated with the extracted features.
        extraction_interval: The frame length for feature extraction.
        sample_rate (int, optional): The sample rate of the audio files. Defaults to 44100 Hz.
        n_mfcc (int, optional): The number of Mel-frequency cepstral coefficients (MFCCs) to extract. Defaults to 13.
        melkwargs (dict, optional): Additional arguments for Mel spectrogram computation.
        n_cqt (int, optional): The number of constant-Q transform (CQT) bins to extract. Defaults to 84.
        n_rms (int, optional): The number of RMS values to extract. Defaults to 20.
        n_zcr (int, optional): The number of zero-crossing rate values to extract. Defaults to 20.
        n_sc (int, optional): The number of spectral centroid values to extract. Defaults to 20.
        n_sb (int, optional): The number of spectral bandwidth values to extract. Defaults to 20.
        n_sr (int, optional): The number of spectral rolloff values to extract. Defaults to 20.
        get_mfcc (bool, optional): Whether to extract MFCC features. Defaults to False.
        get_chroma (bool, optional): Whether to extract chroma features. Defaults to False.
        get_cqt (bool, optional): Whether to extract CQT features. Defaults to False.
        get_rms (bool, optional): Whether to extract RMS features. Defaults to False.
        get_zcr (bool, optional): Whether to extract zero-crossing rate features. Defaults to False.
        get_sc (bool, optional): Whether to extract spectral centroid features. Defaults to False.
        get_sb (bool, optional): Whether to extract spectral bandwidth features. Defaults to False.
        get_sr (bool, optional): Whether to extract spectral rolloff features. Defaults to False.
        

    Returns:
        list: A list containing tensors of extracted features for each audio file.
    """
    filenames = os.listdir(dir_path)
    filenames = [file for file in filenames if not file.startswith(".")]
    extraction_interval = 1 / extraction_interval

    features = []  # List to store the features for all files

    for file in tqdm(filenames, desc=f"Extraction in progress"):
        audio, orig_sample_rate = torchaudio.load(os.path.join(dir_path, file))
        orig_sample_rate_tmp = int(orig_sample_rate / extraction_interval)
        audio_length = int(max(audio[0].shape) / (orig_sample_rate_tmp))

        sample_rate = orig_sample_rate

        # Reduce the audio from stereo to mono if needed
        if audio.shape[0] > 1:
            print(f"Converting stereo audio to mono for {file}...")
            audio = torch.mean(audio, dim=0).reshape(1, -1)

        features_local = []  # List to store the MFCC features for each file

        for i in range(audio_length):
            # Lazy load the audio, one sec at a time, to avoid memory issues
            audio_mono = utils.slicing(
                audio,
                offset=int(sample_rate / extraction_interval * i),
                num_frames=int(sample_rate / extraction_interval * (i + 1)),
            )

            # Get the MFCC features
            if get_mfcc:
                mfcc_features_tmp = utils.extract_mfcc(
                    audio_mono, sample_rate, n_mfcc, melkwargs
                )
            if get_chroma:
                chroma_stft = utils.extract_chroma_stft(audio_mono, sample_rate)
            if get_cqt:
                cqt = utils.extract_cqt(
                    audio_mono, sample_rate=sample_rate, n_cqt=n_cqt
                )
            if get_rms:
                rms = extract_rms(audio_mono, num_values=n_rms)
            if get_zcr:
                zcr = extract_zero_crossing_rate(audio_mono, num_values=n_zcr)
            if get_sc:
                spec_cent = extract_spectral_centroid(audio_mono, sample_rate, num_values=n_sc)
            if get_sb:
                spec_bw = extract_spectral_bandwidth(audio_mono, sample_rate, num_values=n_sb)
            if get_sr:
                rolloff = extract_spectral_rolloff(audio_mono, sample_rate, num_values=n_sr)

            #   spec_cent = extract_spectral_centroid(audio_mono, sample_rate)
            #   spec_bw = extract_spectral_bandwidth(audio_mono, sample_rate)
            #   rolloff = extract_spectral_rolloff(audio_mono, sample_rate)
            #   zcr = extract_zero_crossing_rate(audio_mono)

            # check that no more than one feature is extracted. Else raise an error
            if sum([get_mfcc, get_chroma, get_cqt, get_rms, get_zcr, get_sc, get_sb, get_sr]) != 1:
                raise ValueError(
                    "Exactly one feature must be extracted at a time. Please check the arguments."
                )
            
            # Concatenate the features
            if get_mfcc:
                features_tmp = mfcc_features_tmp
            if get_chroma:
                features_tmp = chroma_stft
            if get_cqt:
                features_tmp = cqt
            if get_rms:
                features_tmp = rms
            if get_zcr:
                features_tmp = zcr
            if get_sc:
                features_tmp = spec_cent
            if get_sb:
                features_tmp = spec_bw
            if get_sr:
                features_tmp = rolloff
            

            features_local.append(features_tmp)

        # If features_local is empty, skip the file
        if features_local == []:
            print(f"No features extracted for {file}. Skipping...")
            continue

        # Stack the MFCC features into a single tensor
        features_local = torch.stack(features_local, dim=0)

        # Attach the label in the last column
        features_local = torch.cat(
            (features_local, torch.ones(features_local.shape[0], 1) * label), dim=1
        )

        # Attach the filename index in the last column
        features_local = torch.cat(
            (
                features_local,
                torch.full((features_local.shape[0], 1), filenames.index(file)),
            ),
            dim=1,
        )

        features.append(features_local)

    print("Finished processing all files.\n")
    return features

# --------------------------------------------------------------------
# --------------------------------------------------------------------


def extract_rms(audio: torch.Tensor, num_values: int) -> torch.Tensor:
    """
    Extract RMS features from an audio file.

    Args:
    - audio (torch.Tensor): The audio file.
    - num_values (int): The number of RMS values to extract.

    Returns:
    - rms (torch.Tensor): The RMS features of the audio file.
    """
    
    num_samples = audio.shape[1]
    hop_length = num_samples // num_values

    
    rms = librosa.feature.rms(y=audio.numpy(), frame_length=hop_length*2, hop_length=hop_length, center=True)
    rms = torch.tensor(rms)
    return rms.reshape(-1)


# --------------------------------------------------------------------
# --------------------------------------------------------------------


def extract_zero_crossing_rate(audio: torch.Tensor, num_values: int) -> torch.Tensor:
    """
    Extract Zero Crossing Rate features from an audio file.

    Args:
    - audio (torch.Tensor): The audio file.
    - num_values (int): The number of Zero Crossing Rate values to extract.

    Returns:
    - zcr (torch.Tensor): The Zero Crossing Rate features of the audio file.
    """
    num_samples = audio.shape[1]
    hop_length = num_samples // num_values
    
    zcr = librosa.feature.zero_crossing_rate(y=audio.numpy(), frame_length=hop_length*2, hop_length=hop_length)
    zcr = torch.tensor(zcr)
    return zcr.reshape(-1)

# --------------------------------------------------------------------
# --------------------------------------------------------------------

def extract_spectral_centroid(audio: torch.Tensor, sample_rate: int, num_values: int) -> torch.Tensor:
    """
    Extract Spectral Centroid features from an audio file.

    Args:
    - audio (torch.Tensor): The audio file.
    - sample_rate (int): The sample rate of the audio file.
    - num_values (int): The number of Spectral Centroid values to extract.

    Returns:
    - spec_cent (torch.Tensor): The Spectral Centroid features of the audio file.
    """
    num_samples = audio.shape[1]
    hop_length = num_samples // num_values
    
    sb = librosa.feature.spectral_centroid(y=audio.numpy(), n_fft=hop_length*2, hop_length=hop_length, sr=sample_rate)
    sb = torch.tensor(sb)
    return sb.reshape(-1)


# --------------------------------------------------------------------
# --------------------------------------------------------------------


def extract_spectral_bandwidth(audio: torch.Tensor, sample_rate: int, num_values:int) -> torch.Tensor:
    """
    Extract Spectral Bandwidth features from an audio file.

    Args:
    - audio (torch.Tensor): The audio file.
    - sample_rate (int): The sample rate of the audio file.
    - num_values (int): The number of Spectral Bandwidth values to extract.

    Returns:
    - spec_bw (torch.Tensor): The Spectral Bandwidth features of the audio file.
    """
    num_samples = audio.shape[1]
    hop_length = num_samples // num_values
    
    sc = librosa.feature.spectral_bandwidth(y=audio.numpy(), n_fft=hop_length*2, hop_length=hop_length, sr=sample_rate)
    sc = torch.tensor(sc)
    return sc.reshape(-1)


# --------------------------------------------------------------------
# --------------------------------------------------------------------


def extract_spectral_rolloff(audio: torch.Tensor, sample_rate: int, num_values:int) -> torch.Tensor:
    """
    Extract Spectral Rolloff features from an audio file.

    Args:
    - audio (torch.Tensor): The audio file.
    - sample_rate (int): The sample rate of the audio file.
    - num_values (int): The number of Spectral Rolloff values to extract.

    Returns:
    - rolloff (torch.Tensor): The Spectral Rolloff features of the audio file.
    """
    num_samples = audio.shape[1]
    hop_length = num_samples // num_values
    
    sr = librosa.feature.spectral_rolloff(y=audio.numpy(), n_fft=hop_length*2, hop_length=hop_length, sr=sample_rate)
    sr = torch.tensor(sr)
    return sr.reshape(-1)

In [5]:
# ------------------------- MFCC -------------------------
#n_mfcc_list = [30, 60, 90, 120]
n_mfcc_list = [30]
names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = INTERVAL
type_ = "mfcc"  # mfcc, mel_spec, spec, cqt
# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

for n_mfcc in n_mfcc_list:
    # Save the features to a file
    storing_name = f"full_data_{interval}s_{sr}hz_{n_mfcc}{type_}"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE

    if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
        print("The features have already been extracted")

    else:
        for i, PATH_ in enumerate(PATHS):

            print(f"Extracting features from {PATH_}")
            features = extract_features(
                PATH_,
                label=i,
                extraction_interval=interval,
                n_mfcc=n_mfcc,
                get_mfcc=True,
                get_chroma=False,
                get_cqt=False,
            )

            # Stack the features into a single tensor
            features = torch.cat(features, dim=0).numpy()
            print(f"The shape of the {PATH_} features tensor is: {features.shape}")

            X = features[:, :-2]
            y = features[:, -2]
            filename = features[:, -1]

            local_dict = {"X": X, "y": y, "filename": filename}

            features_dict[names[i]] = local_dict

        # Save the features to a file
        np.save(FEATURES_RAW_DIR + storing_name, features_dict)

        print("Features extracted and saved")

Extracting features from ../dataset/artifacts_4000/


Extraction in progress:   0%|          | 0/84 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for artifact_2023_44.wav...
Converting stereo audio to mono for artifact_2023_45.wav...
Converting stereo audio to mono for arti

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_36USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for extrahls__201104140118.wav. Skipping...
No features extracted for noisy_3USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_32USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104270459.wav. Skipping...
No features extracted for pitch_34USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for noisy_18USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for pitch_32USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_12USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for pitch_12USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for noisy_38USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104021355.wav. Skipping...
No features extracte

...

No features extracted for murmur__293_1311680805936_B1.wav. Skipping...
No features extracted for murmur__201104021355.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_D.wav. Skipping...
Converting stereo audio to mono for holosystolic_murmur_2023_7.wav...
Converting stereo audio to mono for aortic_stenosis_2023_4.wav...
Converting stereo audio to mono for mitral_valve_prolapse_2023_10.wav...
Converting stereo audio to mono for innocent_murmur_2023_8.wav...
Finished processing all files.

The shape of the ../dataset/murmurs_4000/ features tensor is: (534, 32)
Extractin

Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__232_1308748524018_D1.wav. Skipping...
No features extracted for normal__210_1308162935880_D2.wav. Skipping...
No features extracted for normal__210_1308162935880_D1.wav. Skipping...
No features extracted for normal__143_1306763822290_C.wav. Skipping...
No features extracted for normal__147_1306523973811_C.wav. Skipping...
No features extracted for normal__238_1309194586293_B.wav. Skipping...
No features extracted for normal__190_1308076920011_C1.wav. Skipping...
No features extracted for normal__230_1308595300880_B.wav. Skipping...
No features extracted for normal__267_1309368735165_A.wav. Skipping...
No features extracted for normal__238_1309194586293_A.wav. Skipping...
No features extracted for normal__286_1311170606028_A1.wav. Skipping...
No features extracted for normal__210_1308162935880_B1.wav. Skipping...
No features extracted for normal__252_1309203336604_B.wav. Skipping...
No features extracted for normal__134_1306428161797_C2.wav. Skipping...

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_25USERAUGMENTEDextrastole__237_1308750231222_A.wav. Skipping...
No features extracted for extrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_18USERAUGMENTEDextrastole__202_1308145175747_C2.wav. Skipping...
No features extracted for pitch_4USERAUGMENTEDextrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_33USERAUGMENTEDextrastole__134_1306428161797_C1.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrastoles_4000/ features tensor is: (381, 32)
Features extracted and saved


In [6]:
# ------------------------- CHROMA -------------------------
names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = INTERVAL
n_features = 12
type_ = "chroma"  # mfcc, mel_spec, spec, cqt
# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

for n_mfcc in n_mfcc_list:
    # Save the features to a file
    storing_name = f"full_data_{interval}s_{sr}hz_{n_features}{type_}"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE

    if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
        print("The features have already been extracted")

    else:
        for i, PATH_ in enumerate(PATHS):

            print(f"Extracting features from {PATH_}")
            features = extract_features(
                PATH_,
                label=i,
                extraction_interval=interval,
                n_mfcc=n_mfcc,
                get_mfcc=False,
                get_chroma=True,
                get_cqt=False,
            )

            # Stack the features into a single tensor
            features = torch.cat(features, dim=0).numpy()
            print(f"The shape of the {PATH_} features tensor is: {features.shape}")

            X = features[:, :-2]
            y = features[:, -2]
            filename = features[:, -1]

            local_dict = {"X": X, "y": y, "filename": filename}

            features_dict[names[i]] = local_dict

        # Save the features to a file
        np.save(FEATURES_RAW_DIR + storing_name, features_dict)

        print("Features extracted and saved")

Extracting features from ../dataset/artifacts_4000/


Extraction in progress:   0%|          | 0/84 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for artifact_2023_44.wav...
Converting stereo audio to mono for artifact_2023_45.wav...
Converting stereo audio to mono for arti

/opt/homebrew/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Converting stereo audio to mono for artifact_2023_24.wav...
Converting stereo audio to mono for artifact_2023_2.wav...
Converting stereo audio to mono for artifact_2023_25.wav...
Converting stereo audio to mono for artifact_2023_19.wav...
Converting stereo audio to mono for artifact_2023_33.wav...
Converting stereo audio to mono for artifact_2023_1.wav...
Converting stereo audio to mono for artifact_2023_0.wav...
Converting stereo audio to mono for artifact_2023_26.wav...
Converting stereo audio to mono for artifact_2023_32.wav...
Finished processing all files.

The shape of the ../dataset/artifacts_4000/ features tensor is: (571, 14)
Extracting features from ../dataset/extrahls_4000/


...

No features extracted for murmur__293_1311680805936_B1.wav. Skipping...
No features extracted for murmur__201104021355.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_D.wav. Skipping...
Converting stereo audio to mono for holosystolic_murmur_2023_7.wav...
Converting stereo audio to mono for aortic_stenosis_2023_4.wav...
Converting stereo audio to mono for mitral_valve_prolapse_2023_10.wav...
Converting stereo audio to mono for innocent_murmur_2023_8.wav...
Finished processing all files.

The shape of the ../dataset/murmurs_4000/ features tensor is: (534, 14)
Extractin

Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__232_1308748524018_D1.wav. Skipping...
No features extracted for normal__210_1308162935880_D2.wav. Skipping...
No features extracted for normal__210_1308162935880_D1.wav. Skipping...
No features extracted for normal__143_1306763822290_C.wav. Skipping...
No features extracted for normal__147_1306523973811_C.wav. Skipping...
No features extracted for normal__238_1309194586293_B.wav. Skipping...
No features extracted for normal__190_1308076920011_C1.wav. Skipping...
No features extracted for normal__230_1308595300880_B.wav. Skipping...
No features extracted for normal__267_1309368735165_A.wav. Skipping...
No features extracted for normal__238_1309194586293_A.wav. Skipping...
No features extracted for normal__286_1311170606028_A1.wav. Skipping...
No features extracted for normal__210_1308162935880_B1.wav. Skipping...
No features extracted for normal__252_1309203336604_B.wav. Skipping...
No features extracted for normal__134_1306428161797_C2.wav. Skipping...

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_25USERAUGMENTEDextrastole__237_1308750231222_A.wav. Skipping...
No features extracted for extrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_18USERAUGMENTEDextrastole__202_1308145175747_C2.wav. Skipping...
No features extracted for pitch_4USERAUGMENTEDextrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_33USERAUGMENTEDextrastole__134_1306428161797_C1.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrastoles_4000/ features tensor is: (381, 14)
Features extracted and saved


In [7]:
# ------------------------- CQT -------------------------
# n_cqt_list = [
#     20,
#     30,
#     40,
#     60,
#     70,
# ]
n_cqt_list = [70]

names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = INTERVAL
type_ = "cqt"  # mfcc, mel_spec, spec, cqt
# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

for n_cqt in n_cqt_list:
    # Save the features to a file
    storing_name = f"full_data_{interval}s_{sr}hz_{n_cqt}{type_}"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE

    if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
        print("The features have already been extracted")

    else:
        for i, PATH_ in enumerate(PATHS):

            print(f"Extracting features from {PATH_}")
            features = extract_features(
                PATH_,
                label=i,
                extraction_interval=interval,
                n_cqt=n_cqt,
                get_mfcc=False,
                get_chroma=False,
                get_cqt=True,
            )

            # Stack the features into a single tensor
            features = torch.cat(features, dim=0).numpy()
            print(f"The shape of the {PATH_} features tensor is: {features.shape}")

            X = features[:, :-2]
            y = features[:, -2]
            filename = features[:, -1]

            local_dict = {"X": X, "y": y, "filename": filename}

            features_dict[names[i]] = local_dict

        # Save the features to a file
        np.save(FEATURES_RAW_DIR + storing_name, features_dict)

        print("Features extracted and saved")

Extracting features from ../dataset/artifacts_4000/


Extraction in progress:   0%|          | 0/84 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for artifact_2023_44.wav...
Converting stereo audio to mono for artifact_2023_45.wav...
Converting stereo audio to mono for arti

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_36USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for extrahls__201104140118.wav. Skipping...
No features extracted for noisy_3USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_32USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104270459.wav. Skipping...
No features extracted for pitch_34USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for noisy_18USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for pitch_32USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_12USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for pitch_12USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for noisy_38USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104021355.wav. Skipping...
No features extracte

...

No features extracted for murmur__293_1311680805936_B1.wav. Skipping...
No features extracted for murmur__201104021355.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_D.wav. Skipping...
Converting stereo audio to mono for holosystolic_murmur_2023_7.wav...
Converting stereo audio to mono for aortic_stenosis_2023_4.wav...
Converting stereo audio to mono for mitral_valve_prolapse_2023_10.wav...
Converting stereo audio to mono for innocent_murmur_2023_8.wav...
Finished processing all files.

The shape of the ../dataset/murmurs_4000/ features tensor is: (534, 72)
Extractin

Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__232_1308748524018_D1.wav. Skipping...
No features extracted for normal__210_1308162935880_D2.wav. Skipping...
No features extracted for normal__210_1308162935880_D1.wav. Skipping...
No features extracted for normal__143_1306763822290_C.wav. Skipping...
No features extracted for normal__147_1306523973811_C.wav. Skipping...
No features extracted for normal__238_1309194586293_B.wav. Skipping...
No features extracted for normal__190_1308076920011_C1.wav. Skipping...
No features extracted for normal__230_1308595300880_B.wav. Skipping...
No features extracted for normal__267_1309368735165_A.wav. Skipping...
No features extracted for normal__238_1309194586293_A.wav. Skipping...
No features extracted for normal__286_1311170606028_A1.wav. Skipping...
No features extracted for normal__210_1308162935880_B1.wav. Skipping...
No features extracted for normal__252_1309203336604_B.wav. Skipping...
No features extracted for normal__134_1306428161797_C2.wav. Skipping...

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_25USERAUGMENTEDextrastole__237_1308750231222_A.wav. Skipping...
No features extracted for extrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_18USERAUGMENTEDextrastole__202_1308145175747_C2.wav. Skipping...
No features extracted for pitch_4USERAUGMENTEDextrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_33USERAUGMENTEDextrastole__134_1306428161797_C1.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrastoles_4000/ features tensor is: (381, 72)
Features extracted and saved


In [8]:
# ------------------------- RMS -------------------------
# n_rms_list = [
#     20,
#     40,
#     60,
# ]

n_rms_list = [40]

names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = INTERVAL
type_ = "rms"  # mfcc, mel_spec, spec, cqt, rms

for n_rms in n_rms_list:
    # Save the features to a file
    storing_name = f"full_data_{interval}s_{sr}hz_{n_rms+1}{type_}"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE

    if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
        print("The features have already been extracted")

    else:
        for i, PATH_ in enumerate(PATHS):

            print(f"Extracting features from {PATH_}")
            features = extract_features(
                PATH_,
                label=i,
                extraction_interval=interval,
                n_rms=n_rms,
                get_mfcc=False,
                get_chroma=False,
                get_cqt=False,
                get_rms=True,
            )

            # Stack the features into a single tensor
            features = torch.cat(features, dim=0).numpy()
            print(f"The shape of the {PATH_} features tensor is: {features.shape}")

            X = features[:, :-2]
            y = features[:, -2]
            filename = features[:, -1]

            local_dict = {"X": X, "y": y, "filename": filename}

            features_dict[names[i]] = local_dict

        # Save the features to a file
        np.save(FEATURES_RAW_DIR + storing_name, features_dict)

        print("Features extracted and saved")

Extracting features from ../dataset/artifacts_4000/


Extraction in progress:   0%|          | 0/84 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for artifact_2023_44.wav...
Converting stereo audio to mono for artifact_2023_45.wav...
Converting stereo audio to mono for arti

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_36USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for extrahls__201104140118.wav. Skipping...
No features extracted for noisy_3USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_32USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104270459.wav. Skipping...
No features extracted for pitch_34USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for noisy_18USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for pitch_32USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_12USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for pitch_12USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for noisy_38USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104021355.wav. Skipping...
No features extracte

...

No features extracted for murmur__293_1311680805936_B1.wav. Skipping...
No features extracted for murmur__201104021355.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_D.wav. Skipping...
Converting stereo audio to mono for holosystolic_murmur_2023_7.wav...
Converting stereo audio to mono for aortic_stenosis_2023_4.wav...
Converting stereo audio to mono for mitral_valve_prolapse_2023_10.wav...
Converting stereo audio to mono for innocent_murmur_2023_8.wav...
Finished processing all files.

The shape of the ../dataset/murmurs_4000/ features tensor is: (534, 43)
Extractin

Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__232_1308748524018_D1.wav. Skipping...
No features extracted for normal__210_1308162935880_D2.wav. Skipping...
No features extracted for normal__210_1308162935880_D1.wav. Skipping...
No features extracted for normal__143_1306763822290_C.wav. Skipping...
No features extracted for normal__147_1306523973811_C.wav. Skipping...
No features extracted for normal__238_1309194586293_B.wav. Skipping...
No features extracted for normal__190_1308076920011_C1.wav. Skipping...
No features extracted for normal__230_1308595300880_B.wav. Skipping...
No features extracted for normal__267_1309368735165_A.wav. Skipping...
No features extracted for normal__238_1309194586293_A.wav. Skipping...
No features extracted for normal__286_1311170606028_A1.wav. Skipping...
No features extracted for normal__210_1308162935880_B1.wav. Skipping...
No features extracted for normal__252_1309203336604_B.wav. Skipping...
No features extracted for normal__134_1306428161797_C2.wav. Skipping...

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_25USERAUGMENTEDextrastole__237_1308750231222_A.wav. Skipping...
No features extracted for extrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_18USERAUGMENTEDextrastole__202_1308145175747_C2.wav. Skipping...
No features extracted for pitch_4USERAUGMENTEDextrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_33USERAUGMENTEDextrastole__134_1306428161797_C1.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrastoles_4000/ features tensor is: (381, 43)
Features extracted and saved


In [9]:
# ------------------------- ZCR -------------------------
# n_zcr_list = [
#     20,
#     40,
#     60,
# ]
n_zcr_list = [40]

names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = INTERVAL
type_ = "zcr"  # mfcc, mel_spec, spec, cqt, rms

for n_zcr in n_zcr_list:
    # Save the features to a file
    storing_name = f"full_data_{interval}s_{sr}hz_{n_zcr+1}{type_}"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE

    if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
        print("The features have already been extracted")

    else:
        for i, PATH_ in enumerate(PATHS):

            print(f"Extracting features from {PATH_}")
            features = extract_features(
                PATH_,
                label=i,
                extraction_interval=interval,
                n_zcr=n_zcr,
                get_mfcc=False,
                get_chroma=False,
                get_cqt=False,
                get_rms=False,
                get_zcr=True,
            )

            # Stack the features into a single tensor
            features = torch.cat(features, dim=0).numpy()
            print(f"The shape of the {PATH_} features tensor is: {features.shape}")

            X = features[:, :-2]
            y = features[:, -2]
            filename = features[:, -1]

            local_dict = {"X": X, "y": y, "filename": filename}

            features_dict[names[i]] = local_dict

        # Save the features to a file
        np.save(FEATURES_RAW_DIR + storing_name, features_dict)

        print("Features extracted and saved")

Extracting features from ../dataset/artifacts_4000/


Extraction in progress:   0%|          | 0/84 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for artifact_2023_44.wav...
Converting stereo audio to mono for artifact_2023_45.wav...
Converting stereo audio to mono for arti

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_36USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for extrahls__201104140118.wav. Skipping...
No features extracted for noisy_3USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_32USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104270459.wav. Skipping...
No features extracted for pitch_34USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for noisy_18USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for pitch_32USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_12USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for pitch_12USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for noisy_38USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104021355.wav. Skipping...
No features extracte

...

No features extracted for murmur__293_1311680805936_B1.wav. Skipping...
No features extracted for murmur__201104021355.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_D.wav. Skipping...
Converting stereo audio to mono for holosystolic_murmur_2023_7.wav...
Converting stereo audio to mono for aortic_stenosis_2023_4.wav...
Converting stereo audio to mono for mitral_valve_prolapse_2023_10.wav...
Converting stereo audio to mono for innocent_murmur_2023_8.wav...
Finished processing all files.

The shape of the ../dataset/murmurs_4000/ features tensor is: (534, 43)
Extractin

Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__232_1308748524018_D1.wav. Skipping...
No features extracted for normal__210_1308162935880_D2.wav. Skipping...
No features extracted for normal__210_1308162935880_D1.wav. Skipping...
No features extracted for normal__143_1306763822290_C.wav. Skipping...
No features extracted for normal__147_1306523973811_C.wav. Skipping...
No features extracted for normal__238_1309194586293_B.wav. Skipping...
No features extracted for normal__190_1308076920011_C1.wav. Skipping...
No features extracted for normal__230_1308595300880_B.wav. Skipping...
No features extracted for normal__267_1309368735165_A.wav. Skipping...
No features extracted for normal__238_1309194586293_A.wav. Skipping...
No features extracted for normal__286_1311170606028_A1.wav. Skipping...
No features extracted for normal__210_1308162935880_B1.wav. Skipping...
No features extracted for normal__252_1309203336604_B.wav. Skipping...
No features extracted for normal__134_1306428161797_C2.wav. Skipping...

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_25USERAUGMENTEDextrastole__237_1308750231222_A.wav. Skipping...
No features extracted for extrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_18USERAUGMENTEDextrastole__202_1308145175747_C2.wav. Skipping...
No features extracted for pitch_4USERAUGMENTEDextrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_33USERAUGMENTEDextrastole__134_1306428161797_C1.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrastoles_4000/ features tensor is: (381, 43)
Features extracted and saved


In [10]:
# ------------------------- SPECTRAL CENTROID -------------------------
# n_sc_list = [
#     20,
#     40,
#     60,
# ]
n_sc_list = [40]

names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = INTERVAL
type_ = "sc"  # mfcc, mel_spec, spec, cqt, rms

for n_sc in n_sc_list:
    # Save the features to a file
    storing_name = f"full_data_{interval}s_{sr}hz_{n_sc+1}{type_}"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE

    if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
        print("The features have already been extracted")

    else:
        for i, PATH_ in enumerate(PATHS):

            print(f"Extracting features from {PATH_}")
            features = extract_features(
                PATH_,
                label=i,
                extraction_interval=interval,
                n_sc=n_sc,
                get_mfcc=False,
                get_chroma=False,
                get_cqt=False,
                get_rms=False,
                get_zcr=False,
                get_sc=True,
            )

            # Stack the features into a single tensor
            features = torch.cat(features, dim=0).numpy()
            print(f"The shape of the {PATH_} features tensor is: {features.shape}")

            X = features[:, :-2]
            y = features[:, -2]
            filename = features[:, -1]

            local_dict = {"X": X, "y": y, "filename": filename}

            features_dict[names[i]] = local_dict

        # Save the features to a file
        np.save(FEATURES_RAW_DIR + storing_name, features_dict)

        print("Features extracted and saved")

Extracting features from ../dataset/artifacts_4000/


Extraction in progress:   0%|          | 0/84 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for artifact_2023_44.wav...
Converting stereo audio to mono for artifact_2023_45.wav...
Converting stereo audio to mono for arti

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_36USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for extrahls__201104140118.wav. Skipping...
No features extracted for noisy_3USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_32USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104270459.wav. Skipping...
No features extracted for pitch_34USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for noisy_18USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for pitch_32USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_12USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for pitch_12USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for noisy_38USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104021355.wav. Skipping...
No features extracte

...

No features extracted for murmur__293_1311680805936_B1.wav. Skipping...
No features extracted for murmur__201104021355.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_D.wav. Skipping...
Converting stereo audio to mono for holosystolic_murmur_2023_7.wav...
Converting stereo audio to mono for aortic_stenosis_2023_4.wav...
Converting stereo audio to mono for mitral_valve_prolapse_2023_10.wav...
Converting stereo audio to mono for innocent_murmur_2023_8.wav...
Finished processing all files.

The shape of the ../dataset/murmurs_4000/ features tensor is: (534, 43)
Extractin

Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__232_1308748524018_D1.wav. Skipping...
No features extracted for normal__210_1308162935880_D2.wav. Skipping...
No features extracted for normal__210_1308162935880_D1.wav. Skipping...
No features extracted for normal__143_1306763822290_C.wav. Skipping...
No features extracted for normal__147_1306523973811_C.wav. Skipping...
No features extracted for normal__238_1309194586293_B.wav. Skipping...
No features extracted for normal__190_1308076920011_C1.wav. Skipping...
No features extracted for normal__230_1308595300880_B.wav. Skipping...
No features extracted for normal__267_1309368735165_A.wav. Skipping...
No features extracted for normal__238_1309194586293_A.wav. Skipping...
No features extracted for normal__286_1311170606028_A1.wav. Skipping...
No features extracted for normal__210_1308162935880_B1.wav. Skipping...
No features extracted for normal__252_1309203336604_B.wav. Skipping...
No features extracted for normal__134_1306428161797_C2.wav. Skipping...

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_25USERAUGMENTEDextrastole__237_1308750231222_A.wav. Skipping...
No features extracted for extrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_18USERAUGMENTEDextrastole__202_1308145175747_C2.wav. Skipping...
No features extracted for pitch_4USERAUGMENTEDextrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_33USERAUGMENTEDextrastole__134_1306428161797_C1.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrastoles_4000/ features tensor is: (381, 43)
Features extracted and saved


In [11]:
# ------------------------- SPECTRAL BANDWIDTH -------------------------
# n_sb_list = [
#     20,
#     40,
#     60,
# ]

n_sb_list = [60]

names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = INTERVAL
type_ = "sb"  # mfcc, mel_spec, spec, cqt, rms

for n_sb in n_sb_list:
    # Save the features to a file
    storing_name = f"full_data_{interval}s_{sr}hz_{n_sb+1}{type_}"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE

    if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
        print("The features have already been extracted")

    else:
        for i, PATH_ in enumerate(PATHS):

            print(f"Extracting features from {PATH_}")
            features = extract_features(
                PATH_,
                label=i,
                extraction_interval=interval,
                n_sb=n_sb,
                get_mfcc=False,
                get_chroma=False,
                get_cqt=False,
                get_rms=False,
                get_zcr=False,
                get_sc=False,
                get_sb = True
            )

            # Stack the features into a single tensor
            features = torch.cat(features, dim=0).numpy()
            print(f"The shape of the {PATH_} features tensor is: {features.shape}")

            X = features[:, :-2]
            y = features[:, -2]
            filename = features[:, -1]

            local_dict = {"X": X, "y": y, "filename": filename}

            features_dict[names[i]] = local_dict

        # Save the features to a file
        np.save(FEATURES_RAW_DIR + storing_name, features_dict)

        print("Features extracted and saved")

Extracting features from ../dataset/artifacts_4000/


Extraction in progress:   0%|          | 0/84 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for artifact_2023_44.wav...
Converting stereo audio to mono for artifact_2023_45.wav...
Converting stereo audio to mono for arti

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_36USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for extrahls__201104140118.wav. Skipping...
No features extracted for noisy_3USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_32USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104270459.wav. Skipping...
No features extracted for pitch_34USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for noisy_18USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for pitch_32USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_12USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for pitch_12USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for noisy_38USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104021355.wav. Skipping...
No features extracte

...

No features extracted for murmur__293_1311680805936_B1.wav. Skipping...
No features extracted for murmur__201104021355.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_D.wav. Skipping...
Converting stereo audio to mono for holosystolic_murmur_2023_7.wav...
Converting stereo audio to mono for aortic_stenosis_2023_4.wav...
Converting stereo audio to mono for mitral_valve_prolapse_2023_10.wav...
Converting stereo audio to mono for innocent_murmur_2023_8.wav...
Finished processing all files.

The shape of the ../dataset/murmurs_4000/ features tensor is: (534, 63)
Extractin

Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__232_1308748524018_D1.wav. Skipping...
No features extracted for normal__210_1308162935880_D2.wav. Skipping...
No features extracted for normal__210_1308162935880_D1.wav. Skipping...
No features extracted for normal__143_1306763822290_C.wav. Skipping...
No features extracted for normal__147_1306523973811_C.wav. Skipping...
No features extracted for normal__238_1309194586293_B.wav. Skipping...
No features extracted for normal__190_1308076920011_C1.wav. Skipping...
No features extracted for normal__230_1308595300880_B.wav. Skipping...
No features extracted for normal__267_1309368735165_A.wav. Skipping...
No features extracted for normal__238_1309194586293_A.wav. Skipping...
No features extracted for normal__286_1311170606028_A1.wav. Skipping...
No features extracted for normal__210_1308162935880_B1.wav. Skipping...
No features extracted for normal__252_1309203336604_B.wav. Skipping...
No features extracted for normal__134_1306428161797_C2.wav. Skipping...

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_25USERAUGMENTEDextrastole__237_1308750231222_A.wav. Skipping...
No features extracted for extrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_18USERAUGMENTEDextrastole__202_1308145175747_C2.wav. Skipping...
No features extracted for pitch_4USERAUGMENTEDextrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_33USERAUGMENTEDextrastole__134_1306428161797_C1.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrastoles_4000/ features tensor is: (381, 63)
Features extracted and saved


In [12]:
# ------------------------- SPECTRAL ROLLOFF -------------------------
# n_sr_list = [
#     20,
#     40,
#     60,
# ]

n_sr_list = [40]

names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = INTERVAL
type_ = "sr"  # mfcc, mel_spec, spec, cqt, rms

for n_sr in n_sr_list:
    # Save the features to a file
    storing_name = f"full_data_{interval}s_{sr}hz_{n_sr+1}{type_}"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE

    if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
        print("The features have already been extracted")

    else:
        for i, PATH_ in enumerate(PATHS):

            print(f"Extracting features from {PATH_}")
            features = extract_features(
                PATH_,
                label=i,
                extraction_interval=interval,
                n_sr=n_sr,
                get_mfcc=False,
                get_chroma=False,
                get_cqt=False,
                get_rms=False,
                get_zcr=False,
                get_sc=False,
                get_sb = False,
                get_sr=True,
            )

            # Stack the features into a single tensor
            features = torch.cat(features, dim=0).numpy()
            print(f"The shape of the {PATH_} features tensor is: {features.shape}")

            X = features[:, :-2]
            y = features[:, -2]
            filename = features[:, -1]

            local_dict = {"X": X, "y": y, "filename": filename}

            features_dict[names[i]] = local_dict

        # Save the features to a file
        np.save(FEATURES_RAW_DIR + storing_name, features_dict)

        print("Features extracted and saved")

Extracting features from ../dataset/artifacts_4000/


Extraction in progress:   0%|          | 0/84 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for artifact_2023_44.wav...
Converting stereo audio to mono for artifact_2023_45.wav...
Converting stereo audio to mono for arti

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_36USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for extrahls__201104140118.wav. Skipping...
No features extracted for noisy_3USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_32USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104270459.wav. Skipping...
No features extracted for pitch_34USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for noisy_18USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for pitch_32USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for noisy_12USERAUGMENTEDextrahls__201104140118.wav. Skipping...
No features extracted for pitch_12USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for noisy_38USERAUGMENTEDextrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104021355.wav. Skipping...
No features extracte

...

No features extracted for murmur__293_1311680805936_B1.wav. Skipping...
No features extracted for murmur__201104021355.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_D.wav. Skipping...
Converting stereo audio to mono for holosystolic_murmur_2023_7.wav...
Converting stereo audio to mono for aortic_stenosis_2023_4.wav...
Converting stereo audio to mono for mitral_valve_prolapse_2023_10.wav...
Converting stereo audio to mono for innocent_murmur_2023_8.wav...
Finished processing all files.

The shape of the ../dataset/murmurs_4000/ features tensor is: (534, 43)
Extractin

Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__232_1308748524018_D1.wav. Skipping...
No features extracted for normal__210_1308162935880_D2.wav. Skipping...
No features extracted for normal__210_1308162935880_D1.wav. Skipping...
No features extracted for normal__143_1306763822290_C.wav. Skipping...
No features extracted for normal__147_1306523973811_C.wav. Skipping...
No features extracted for normal__238_1309194586293_B.wav. Skipping...
No features extracted for normal__190_1308076920011_C1.wav. Skipping...
No features extracted for normal__230_1308595300880_B.wav. Skipping...
No features extracted for normal__267_1309368735165_A.wav. Skipping...
No features extracted for normal__238_1309194586293_A.wav. Skipping...
No features extracted for normal__286_1311170606028_A1.wav. Skipping...
No features extracted for normal__210_1308162935880_B1.wav. Skipping...
No features extracted for normal__252_1309203336604_B.wav. Skipping...
No features extracted for normal__134_1306428161797_C2.wav. Skipping...

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_25USERAUGMENTEDextrastole__237_1308750231222_A.wav. Skipping...
No features extracted for extrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_18USERAUGMENTEDextrastole__202_1308145175747_C2.wav. Skipping...
No features extracted for pitch_4USERAUGMENTEDextrastole__190_1308076920011_C.wav. Skipping...
No features extracted for speed_33USERAUGMENTEDextrastole__134_1306428161797_C1.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrastoles_4000/ features tensor is: (381, 43)
Features extracted and saved


### All Features Extraction

- **MFCC (Mel-Frequency Cepstral Coefficients)**: n_mfcc chosen by the user
- **Chroma STFT**: 12 coefficients
- **CQT (Constant-Q Transform)**: n_cqt chosen by the user
- **RMS (Root Mean Square) Energy**: 1 coefficient
- **Spectral Centroid**: 1 coefficient
- **Spectral Bandwidth**: 1 coefficient
- **Spectral Roll-off**: 1 coefficient
- **Zero Crossing Rate**: 1 coefficient


In [ ]:
# FEATURES EXTRACTION
names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = 1
n_mfcc = 30  # 30, 120
n_cqt = 70  # 30, 70

# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

# Save the features to a file
storing_name = f"full_data_{interval}s_{sr}hz_{n_mfcc}mfcc_{n_cqt}cqt_12chroma"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE


if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
    print("The features have already been extracted")

else:
    for i, PATH_ in enumerate(PATHS):

        print(f"Extracting features from {PATH_}")
        features = utils.extract_features(
            PATH_, label=i, frame_length=interval, n_mfcc=n_mfcc, n_cqt=n_cqt
        )

        # Stack the features into a single tensor
        features = torch.cat(features, dim=0).numpy()
        print(f"The shape of the {PATH_} features tensor is: {features.shape}")

        X = features[:, :-2]
        y = features[:, -2]
        filename = features[:, -1]

        local_dict = {"X": X, "y": y, "filename": filename}

        features_dict[names[i]] = local_dict

    # Save the features to a file
    np.save(FEATURES_RAW_DIR + storing_name, features_dict)

    print("Features extracted and saved")

In [ ]:
# load the features and check the shape
features_dict = np.load(
    FEATURES_RAW_DIR + storing_name + ".npy", allow_pickle=True
).item()
for key in features_dict.keys():
    print(f'The shape of the {key} features tensor is: {features_dict[key]["X"].shape}')
    print(f'The shape of the {key} labels tensor is: {features_dict[key]["y"].shape}')
    print(
        f'The shape of the {key} filenames tensor is: {features_dict[key]["filename"].shape}'
    )
    print("-----------------------------------------------")

In [ ]:
s = 0
for key in features_dict.keys():
    s += features_dict[key]["X"].shape[0]
s